## DataCollection

In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic Python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

## Make repository for Dataset

In [2]:
import os

DATASET_DIR = 'dataset/no_tag' 
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')


Directories not created because they already exist


## Controller set up

In [3]:
controller = widgets.Controller(index=0)

display(controller)

Controller()

## Car set up

In [ ]:
from jetbot import Robot
import traitlets

robot = Robot()


In [ ]:
#move forward and back
move_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'),(robot.right_motor, 'value'), transform=lambda x: -x)
import traitlets

# 左に曲がるためのリンク（左モータの速度を高くし、右モータを低くする）
left_turn_link = traitlets.dlink((controller.axes[0], 'value'), (robot.left_motor, 'value'), transform=lambda x: x if x > 0 else 0)
right_turn_link = traitlets.dlink((controller.axes[0], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x if x < 0 else 0)

# これにより、左スティックの左右の動きが左右のモータの速度に直接影響を与えるようになります。

    


## Camera set up

In [4]:
camera = Camera()

## GUI set up

In [5]:
widget_width = camera.width
widget_height = camera.height

image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
target_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)


def display_xy(camera_image):
    image = np.copy(camera_image)
    jpeg_image = bgr8_to_jpeg(image)
    
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value,r_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        free_count.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[5].observe(save_snapshot, names='value')



button_layout = widgets.Layout(width='128px', height='64px')

free_button = widgets.Button(description='add image', button_style='success', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir('dataset/sponge')))

# free_button.on_click(lambda x: save_snapshot)
free_button.on_click(lambda x: save_snapshot(x))

In [6]:
img_box = widgets.HBox([image_widget, target_widget])
button_box = widgets.HBox([free_count, free_button])

DirectionalLink(source=(Axis(value=-1.5259021896696368e-05), 'value'), target=(FloatSlider(value=0.0, descript…

## Lets collect data

In [8]:
display(widgets.HBox([img_box,button_box]))

FloatSlider(value=1.5259021896696368e-05, description='x', max=1.5, min=-1.5, step=0.001)

FloatSlider(value=-1.5259021896696368e-05, description='y', max=1.6, min=-1.6, step=0.001)

FloatSlider(value=-1.5259021896696368e-05, description='r', max=2.0, min=-2.0, step=0.001)

In [ ]:
camera.stop()